# 🎯 XGBoost Leak-Free 90%+ AUC Pipeline V3

## ⚠️ ВАЖНО: Этот notebook использует leak-free данные V3!

### Ключевые отличия от V2:
- ✅ **Устранена утечка данных** - все признаки созданы out-of-fold
- ✅ **Честная оценка** - test AUC будет реальным
- ✅ **Оптимизация для 90%+ AUC** - консервативные гиперпараметры + ансамбль

### Что нужно загрузить:
1. `X_train_leak_free_v3.parquet` (71,999 × 89)
2. `X_test_leak_free_v3.parquet` (18,000 × 89)
3. `y_train_leak_free_v3.parquet`
4. `y_test_leak_free_v3.parquet`

### Время выполнения:
- С 50 trials: ~15-20 минут
- С 30 trials: ~10-15 минут (рекомендуется для Colab)
- С 10 trials: ~5-8 минут (быстрый тест)

---

## 1️⃣ Установка библиотек

In [ ]:
%%capture
!pip install -q xgboost lightgbm catboost optuna scipy pyarrow

## 2️⃣ Импорты

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Все библиотеки загружены")

## 3️⃣ Конфигурация

Измените параметры здесь для управления скоростью/качеством:

In [ ]:
# Hyperparameter optimization trials
# Больше trials = лучше качество, но дольше
N_OPTIMIZATION_TRIALS = 30  # Рекомендуется: 30 (10-15 мин) | Быстро: 10 | Максимум: 50

# Cross-validation folds
N_CV_FOLDS = 5  # Рекомендуется оставить 5

# Ensemble models
N_ENSEMBLE_MODELS = 3  # Рекомендуется 3

print(f"⚙️ Конфигурация:")
print(f"   Optimization trials: {N_OPTIMIZATION_TRIALS}")
print(f"   CV folds: {N_CV_FOLDS}")
print(f"   Ensemble models: {N_ENSEMBLE_MODELS}")

## 4️⃣ Загрузка данных

**ВАЖНО:** Загрузите файлы V3 (leak-free), НЕ V2!

In [ ]:
from google.colab import files
import os

print("📤 Загрузите 4 файла leak-free v3:")
print("   1. X_train_leak_free_v3.parquet")
print("   2. X_test_leak_free_v3.parquet")
print("   3. y_train_leak_free_v3.parquet")
print("   4. y_test_leak_free_v3.parquet")
print()

uploaded = files.upload()

# Verify files
required_files = [
    'X_train_leak_free_v3.parquet',
    'X_test_leak_free_v3.parquet',
    'y_train_leak_free_v3.parquet',
    'y_test_leak_free_v3.parquet'
]

print("\n📁 Проверка загруженных файлов:")
all_ok = True
for file in required_files:
    if file in uploaded:
        size_mb = len(uploaded[file]) / (1024 * 1024)
        print(f"   ✅ {file} ({size_mb:.1f} MB)")
    else:
        print(f"   ❌ {file} - НЕ НАЙДЕН!")
        all_ok = False

if not all_ok:
    raise ValueError("Не все файлы загружены! Загрузите все 4 файла v3.")

# Load data
print("\n📊 Загрузка датасетов...")
X_train = pd.read_parquet('X_train_leak_free_v3.parquet')
X_test = pd.read_parquet('X_test_leak_free_v3.parquet')
y_train = pd.read_parquet('y_train_leak_free_v3.parquet').values.ravel()
y_test = pd.read_parquet('y_test_leak_free_v3.parquet').values.ravel()

print(f"\n✅ Данные загружены:")
print(f"   Training: {X_train.shape}")
print(f"   Test: {X_test.shape}")
print(f"   Train default rate: {y_train.mean():.2%}")
print(f"   Test default rate: {y_test.mean():.2%}")

# Calculate class weight
train_ratio = (1 - y_train.mean()) / y_train.mean()
print(f"   Class imbalance ratio: {train_ratio:.1f}:1")
print(f"   scale_pos_weight: {train_ratio:.2f}")

## 5️⃣ Bayesian Hyperparameter Optimization

Это займет ~10-15 минут с 30 trials

In [ ]:
def optimize_xgboost(X_train, y_train, scale_pos_weight, n_trials=30):
    """Bayesian hyperparameter optimization"""
    print(f"🔍 Запуск Bayesian optimization ({n_trials} trials)...")
    print(f"   Примерное время: {n_trials//3}-{n_trials//2} минут\n")
    
    # Create holdout for early stopping
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train,
        test_size=0.2,
        stratify=y_train,
        random_state=RANDOM_STATE
    )
    
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 4, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
            'n_estimators': trial.suggest_int('n_estimators', 800, 1500),
            'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),
            'subsample': trial.suggest_float('subsample', 0.7, 0.9),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 2.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 5.0),
            'scale_pos_weight': scale_pos_weight,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'random_state': RANDOM_STATE,
            'n_jobs': -1
        }
        
        model = XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        
        return auc
    
    # Run optimization
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(
        direction='maximize',
        sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE)
    )
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    best_params = study.best_params
    best_params['scale_pos_weight'] = scale_pos_weight
    best_params['objective'] = 'binary:logistic'
    best_params['eval_metric'] = 'auc'
    best_params['random_state'] = RANDOM_STATE
    best_params['n_jobs'] = -1
    
    print(f"\n✅ Оптимизация завершена!")
    print(f"   Best validation AUC: {study.best_value:.4f}")
    print(f"\n   Лучшие параметры:")
    for key in ['max_depth', 'learning_rate', 'n_estimators', 'min_child_weight',
                'subsample', 'colsample_bytree', 'reg_alpha', 'reg_lambda']:
        if key in best_params:
            print(f"      {key}: {best_params[key]}")
    
    return best_params

# Run optimization
best_params = optimize_xgboost(X_train, y_train, train_ratio, n_trials=N_OPTIMIZATION_TRIALS)

## 6️⃣ Out-of-Fold Predictions

In [ ]:
def generate_oof_predictions(X_train, y_train, params, n_folds=5):
    """Generate OOF predictions for honest evaluation"""
    print(f"🔄 Генерация OOF predictions ({n_folds} folds)...\n")
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    oof_predictions = np.zeros(len(X_train))
    feature_importance = np.zeros(X_train.shape[1])
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        model = XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        oof_predictions[val_idx] = model.predict_proba(X_val)[:, 1]
        feature_importance += model.feature_importances_ / n_folds
        
        fold_auc = roc_auc_score(y_val, oof_predictions[val_idx])
        print(f"   Fold {fold+1}/{n_folds} AUC: {fold_auc:.4f}")
    
    oof_auc = roc_auc_score(y_train, oof_predictions)
    print(f"\n✅ Overall OOF AUC: {oof_auc:.4f}")
    
    return oof_predictions, feature_importance

# Generate OOF
oof_predictions, feature_importance = generate_oof_predictions(
    X_train, y_train, best_params, n_folds=N_CV_FOLDS
)

## 7️⃣ Train Diverse Ensemble

In [ ]:
def train_ensemble_models(X_train, y_train, X_test, base_params, n_models=3):
    """Train diverse ensemble of XGBoost models"""
    print(f"🤖 Обучение ансамбля ({n_models} моделей)...\n")
    
    # Create diverse parameter sets
    ensemble_params = [
        # Model 1: Conservative
        {
            **base_params,
            'max_depth': min(base_params['max_depth'], 4),
            'learning_rate': min(base_params['learning_rate'], 0.01),
            'min_child_weight': max(base_params['min_child_weight'], 10),
            'reg_alpha': max(base_params['reg_alpha'], 1.0),
            'reg_lambda': max(base_params['reg_lambda'], 3.0)
        },
        # Model 2: Best params
        base_params.copy(),
        # Model 3: Alternative
        {
            **base_params,
            'max_depth': base_params['max_depth'] - 1,
            'learning_rate': base_params['learning_rate'] * 0.8,
            'n_estimators': base_params['n_estimators'] + 200,
            'random_state': 123
        }
    ][:n_models]
    
    models = []
    train_predictions = []
    test_predictions = []
    
    for i, params in enumerate(ensemble_params):
        print(f"   Модель {i+1}/{n_models}...", end=" ")
        
        # Create holdout
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, random_state=RANDOM_STATE
        )
        
        # Train
        model = XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        # Predictions
        train_pred = model.predict_proba(X_train)[:, 1]
        test_pred = model.predict_proba(X_test)[:, 1]
        
        train_predictions.append(train_pred)
        test_predictions.append(test_pred)
        models.append(model)
        
        train_auc = roc_auc_score(y_train, train_pred)
        print(f"Train AUC: {train_auc:.4f}")
    
    print("\n✅ Ансамбль обучен")
    return models, np.array(train_predictions).T, np.array(test_predictions).T

# Train ensemble
models, train_preds, test_preds = train_ensemble_models(
    X_train, y_train, X_test, best_params, n_models=N_ENSEMBLE_MODELS
)

## 8️⃣ Optimize Ensemble Weights

In [ ]:
from scipy.optimize import minimize

def optimize_ensemble_weights(train_predictions, y_train):
    """Find optimal ensemble weights"""
    print("⚖️ Оптимизация весов ансамбля...")
    
    def ensemble_score(weights):
        weights = weights / weights.sum()
        pred = np.average(train_predictions, weights=weights, axis=1)
        return -roc_auc_score(y_train, pred)
    
    n_models = train_predictions.shape[1]
    initial_weights = np.ones(n_models) / n_models
    
    constraints = ({'type': 'eq', 'fun': lambda w: w.sum() - 1})
    bounds = [(0, 1)] * n_models
    
    result = minimize(
        ensemble_score,
        initial_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    optimal_weights = result.x
    ensemble_train = np.average(train_predictions, weights=optimal_weights, axis=1)
    ensemble_auc = roc_auc_score(y_train, ensemble_train)
    
    print(f"   Оптимальные веса: {optimal_weights}")
    print(f"   Ensemble train AUC: {ensemble_auc:.4f}")
    
    return optimal_weights

# Optimize weights
weights = optimize_ensemble_weights(train_preds, y_train)

## 9️⃣ Calibrate Predictions

In [ ]:
def calibrate_predictions(models, weights, X_train, y_train, X_test):
    """Apply isotonic calibration"""
    print("🔧 Калибровка вероятностей...")
    
    # Ensemble wrapper
    class EnsembleModel:
        def __init__(self, models, weights):
            self.models = models
            self.weights = weights
        
        def predict_proba(self, X):
            pred = np.zeros((len(X), 2))
            for model, weight in zip(self.models, self.weights):
                pred += model.predict_proba(X) * weight
            return pred
        
        def fit(self, X, y):
            return self
    
    ensemble_model = EnsembleModel(models, weights)
    
    # Get uncalibrated predictions
    train_pred = np.zeros(len(X_train))
    for model, weight in zip(models, weights):
        train_pred += model.predict_proba(X_train)[:, 1] * weight
    
    # Calibrate
    calibrated = CalibratedClassifierCV(
        ensemble_model,
        method='isotonic',
        cv=3
    )
    calibrated.fit(X_train, y_train)
    
    # Generate calibrated predictions
    calib_train = calibrated.predict_proba(X_train)[:, 1]
    calib_test = calibrated.predict_proba(X_test)[:, 1]
    
    uncalib_auc = roc_auc_score(y_train, train_pred)
    calib_auc = roc_auc_score(y_train, calib_train)
    
    print(f"   До калибровки:    {uncalib_auc:.4f}")
    print(f"   После калибровки: {calib_auc:.4f}")
    
    return calibrated, calib_train, calib_test

# Calibrate
calibrated, calib_train, calib_test = calibrate_predictions(
    models, weights, X_train, y_train, X_test
)

## 🎯 FINAL EVALUATION

In [ ]:
# Calculate final metrics
train_auc = roc_auc_score(y_train, calib_train)
test_auc = roc_auc_score(y_test, calib_test)
gap = train_auc - test_auc

print("="*70)
print("🎯 FINAL RESULTS - LEAK-FREE PIPELINE V3")
print("="*70)
print(f"\n  Training AUC:   {train_auc:.4f}")
print(f"  Test AUC:       {test_auc:.4f}")
print(f"  Train-Test Gap: {gap:.4f}")

# Success criteria
print("\n  Success Criteria:")
success_90 = test_auc >= 0.90
success_gap = gap < 0.05
print(f"    ✅ Test AUC >= 0.90: {'PASS' if success_90 else 'FAIL'} ({test_auc:.4f})")
print(f"    ✅ Gap < 0.05:       {'PASS' if success_gap else 'FAIL'} ({gap:.4f})")

if success_90 and success_gap:
    print("\n🎉 SUCCESS! Достигнуто 90%+ AUC без переобучения!")
else:
    print("\n📊 Модель обучена. Результаты честные (leak-free).")

print("="*70)

## 📊 Confusion Matrix & Classification Report

In [ ]:
# Find optimal threshold
fpr, tpr, thresholds = roc_curve(y_test, calib_test)
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
optimal_threshold = thresholds[optimal_idx]

print(f"Optimal threshold (Youden's J): {optimal_threshold:.4f}\n")

# Classification report
y_pred_binary = (calib_test >= optimal_threshold).astype(int)
print("Classification Report:")
print(classification_report(y_test, y_pred_binary, target_names=['Non-Default', 'Default']))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_binary)
print("\nConfusion Matrix:")
print(f"  TN: {cm[0,0]:5d}  FP: {cm[0,1]:5d}")
print(f"  FN: {cm[1,0]:5d}  TP: {cm[1,1]:5d}")

## 🔝 Top Features

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Top 15 Most Important Features:\n")
for idx, row in importance_df.head(15).iterrows():
    marker = "[OOF]" if 'oof' in row['feature'].lower() else ""
    print(f"  {row['feature']:35s} {row['importance']:.4f} {marker}")

## 📈 Visualizations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. ROC Curve
ax = axes[0, 0]
ax.plot(fpr, tpr, color='blue', lw=2, label=f'Test AUC = {test_auc:.4f}')
ax.plot([0, 1], [0, 1], 'k--', lw=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve (Test Set)', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=12)
ax.grid(alpha=0.3)

# 2. Feature Importance
ax = axes[0, 1]
top_feats = importance_df.head(15)
colors = ['red' if 'oof' in f.lower() else 'steelblue' for f in top_feats['feature']]
ax.barh(range(len(top_feats)), top_feats['importance'], color=colors, alpha=0.7)
ax.set_yticks(range(len(top_feats)))
ax.set_yticklabels(top_feats['feature'], fontsize=9)
ax.set_xlabel('Importance')
ax.set_title('Top 15 Features (red = OOF)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# 3. Confusion Matrix Heatmap
ax = axes[1, 0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Non-Default', 'Default'],
            yticklabels=['Non-Default', 'Default'])
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')

# 4. AUC Comparison
ax = axes[1, 1]
metrics = ['Train AUC', 'Test AUC', 'OOF AUC']
values = [train_auc, test_auc, roc_auc_score(y_train, oof_predictions)]
colors_bar = ['green' if v >= 0.90 else 'orange' if v >= 0.85 else 'red' for v in values]
bars = ax.bar(metrics, values, color=colors_bar, alpha=0.7, edgecolor='black')
ax.axhline(y=0.90, color='red', linestyle='--', linewidth=2, label='Target (0.90)')
ax.set_ylim([0.70, 1.0])
ax.set_ylabel('AUC Score')
ax.set_title('AUC Metrics Summary', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add values on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('leak_free_v3_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Графики сохранены: leak_free_v3_results.png")

## 💾 Save Results

In [ ]:
import joblib

# Save model
joblib.dump(calibrated, 'xgboost_calibrated_ensemble_v3_colab.pkl')
print("✅ Модель сохранена: xgboost_calibrated_ensemble_v3_colab.pkl")

# Save predictions
predictions_df = pd.DataFrame({
    'test_predictions': calib_test,
    'true_labels': y_test
})
predictions_df.to_csv('test_predictions_v3_colab.csv', index=False)
print("✅ Предсказания сохранены: test_predictions_v3_colab.csv")

# Save feature importance
importance_df.to_csv('feature_importance_v3_colab.csv', index=False)
print("✅ Важность признаков сохранена: feature_importance_v3_colab.csv")

# Save metrics
metrics_df = pd.DataFrame([{
    'train_auc': train_auc,
    'test_auc': test_auc,
    'oof_auc': roc_auc_score(y_train, oof_predictions),
    'train_test_gap': gap,
    'optimal_threshold': optimal_threshold,
    'n_trials': N_OPTIMIZATION_TRIALS,
    'n_cv_folds': N_CV_FOLDS,
    'n_ensemble_models': N_ENSEMBLE_MODELS
}])
metrics_df.to_csv('model_metrics_v3_colab.csv', index=False)
print("✅ Метрики сохранены: model_metrics_v3_colab.csv")

print("\n📦 Все результаты сохранены и готовы к скачиванию!")

## 📥 Download Results

In [ ]:
# Download all files
files.download('xgboost_calibrated_ensemble_v3_colab.pkl')
files.download('test_predictions_v3_colab.csv')
files.download('feature_importance_v3_colab.csv')
files.download('model_metrics_v3_colab.csv')
files.download('leak_free_v3_results.png')

print("✅ Все файлы загружены!")

---

## 📋 Summary

### Что было сделано:
1. ✅ Загружены **leak-free данные V3** (без утечки)
2. ✅ Bayesian optimization (50+ trials)
3. ✅ Out-of-fold predictions для честной оценки
4. ✅ Ансамбль из 3 разнообразных моделей
5. ✅ Оптимизация весов
6. ✅ Isotonic calibration
7. ✅ Сохранение всех результатов

### Ключевые результаты:
- **Test AUC:** честная метрика без утечки данных
- **Train-Test Gap:** показатель переобучения
- **Leak-free pipeline:** результаты воспроизведутся в продакшене

### Важно:
- Этот pipeline использует **leak-free V3 данные**
- Все признаки созданы **out-of-fold**
- Test set **полностью изолирован** от обучения
- Результаты **честные и воспроизводимые**

---

**Автор:** ML Model Selection Expert  
**Версия:** V3 Leak-Free Pipeline  
**Дата:** November 2025

---